In [1]:
from importlib import reload
import json
import pretty_midi
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 422091078071127324
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13336856610618758248
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5588779008
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11742072890166847630
physical_device_desc: "device: 0, name: Tesla K20Xm, pci bus id: 0000:00:06.0, compute capability: 3.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 152799329061713978
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
try:
    reload(midiMethods)
    from modules.midiMethods import *
except:
    print('reload failed')
    import modules.midiMethods as midiMethods
    from modules.midiMethods import *
try:
    reload(dataMethods)
    from modules.dataMethods import *
except:
    print('reload failed')
    import modules.dataMethods as dataMethods
    from modules.dataMethods import *
import modules.mlClasses as mlClasses 


In [37]:
import modules.models as models
reload(models)
import modules.mlClasses as mlClasses
reload(mlClasses)

<module 'modules.mlClasses' from 'C:\\Users\\Andrew\\Documents\\mlprojects\\380-music-representation\\modules\\mlClasses.py'>

In [175]:
# test out my function for going to note_bin representation
pm = pretty_midi.PrettyMIDI('midi/basic_test.mid')
noteB2 = pm2note_bin(pm)
print(noteB2)

[[27, 0, 0, 3, 19, 27], [0, 0, 0, 3, 20, 28], [5, 3, 35, 3, 14, 25], [32, 0, 0, 3, 16, 26], [2, 3, 27, 0, 21, 17], [34, 0, 0, 1, 13, 19], [3, 0, 42, 0, 20, 11], [5, 0, 38, 0, 21, 11], [32, 0, 3, 1, 20, 13], [2, 0, 42, 0, 28, 9], [7, 0, 59, 2, 16, 8], [31, 0, 2, 2, 15, 5]]


In [75]:
# if needed, read in filenames...
maestro = pd.read_csv('training_data/maestro-v2.0.0withPeriod.csv', index_col=0)
filenames = list(maestro['midi_filename'])


In [ ]:
# or process data set
data_path = 'C:/Users\Andrew/Documents/mlprojects/Datasets/MaestroV2.00/maestro-v2.0.0/'
examples = files2note_bin_examples(data_path, skip=1)
print(len(examples))

In [322]:
# store examples in json format
with open('training_data/training_data_note_bin_complete.json', 'w') as f:
    json.dump(examples, f)

In [5]:
# or read them in
with open('training_data/training_data_note_bin_complete.json', 'r') as f:
    examples = json.load(f)

In [38]:
# build simple model
# excellent example of recurrent model here https://www.tensorflow.org/tutorials/text/text_generation
hidden_state = 256
lstm_layers = 2
model2 = models.create_model1(hidden_state_size=hidden_state, lstm_layers=lstm_layers)
training_generator = mlClasses.DataGenerator(examples)

Model: "2layerLSTM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128, 6)]     0                                            
__________________________________________________________________________________________________
lambda_18 (Lambda)              (None, 128, 88)      0           input_4[0][0]                    
__________________________________________________________________________________________________
lambda_19 (Lambda)              (None, 128, 10)      0           input_4[0][0]                    
__________________________________________________________________________________________________
lambda_20 (Lambda)              (None, 128, 60)      0           input_4[0][0]                    
_________________________________________________________________________________________

In [ ]:
model2.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# checkpoint = tf.keras.callbacks.ModelCheckpoint("weights/model1/{epoch:02d}-{train_loss:.2f}.hdf5", monitor='train_loss', verbose=2, save_best_only=True, save_weights_only=True)
history = model2.fit_generator(training_generator, epochs=1)
model2.save_weights('weights/note_bin/model2_10epochs256state2layer.h5')

 34/803 [>.............................] - ETA: 47:31 - loss: 15.5474 - dense_18_loss: 4.1541 - dense_19_loss: 0.3063 - dense_20_loss: 3.2233 - dense_21_loss: 1.4997 - dense_22_loss: 3.3131 - dense_23_loss: 3.0508 - dense_18_accuracy: 0.0242 - dense_19_accuracy: 0.9585 - dense_20_accuracy: 0.1783 - dense_21_accuracy: 0.5788 - dense_22_accuracy: 0.0956 - dense_23_accuracy: 0.0740

In [310]:
[Y[:,:,i] for i in range(6)][4]

array([[28, 13, 13, ..., 13, 15, 14],
       [14, 15, 22, ..., 11,  4, 25],
       [17, 22, 18, ..., 18, 17, 10],
       ...,
       [11, 11, 29, ..., 13,  3,  2],
       [ 0, 29, 28, ..., 17, 11,  9],
       [ 9,  8,  3, ..., 29, 29, 29]])

In [31]:
# This works just like this:
try:
    reload(models)
except:
    print('reload failed')
    import modules.models as models

In [8]:
# tf.train.latest_checkpoint('weights\first_attempt')

prediction_model = models.create_model1(batch_size=1, stateful=True, hidden_state_size=hidden_state)

Model: "1layerLSTM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(1, 128, 6)]        0                                            
__________________________________________________________________________________________________
lambda_6 (Lambda)               (1, 128, 88)         0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_7 (Lambda)               (1, 128, 10)         0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_8 (Lambda)               (1, 128, 60)         0           input_2[0][0]                    
_________________________________________________________________________________________

In [33]:
prediction_model.load_weights('weights/note_bin/model2_10epochs256state.h5')
temperatures = [2.2, 0.4, 0.5, 0.5, 0.5, 0.5]
input_note = [44,0,5,3,3,20]
new_seq = models.generate_music(prediction_model, temperatures, input_note)
ns = [[int(a) for a in note] for note in new_seq]
print(ns)
pm = note_bin2pm(ns)
pm.write(f'midi/model2/{temperatures}b.mid')

tf.Tensor([[[44  0  5  3  3 20]]], shape=(1, 1, 6), dtype=int32)
[[44, 0, 5, 3, 3, 20], [23, 3, 31, 14, 10, 10], [50, 0, 15, 10, 16, 17], [11, 5, 37, 4, 4, 7], [75, 4, 3, 13, 10, 17], [27, 9, 55, 7, 20, 5], [3, 6, 37, 10, 8, 19], [35, 9, 18, 9, 14, 7], [66, 4, 55, 6, 23, 11], [34, 5, 29, 0, 12, 9], [54, 1, 39, 0, 26, 21], [38, 0, 33, 3, 20, 28], [76, 0, 27, 5, 6, 1], [2, 0, 34, 17, 5, 20], [65, 0, 8, 12, 17, 25], [68, 2, 9, 0, 18, 12], [52, 8, 14, 13, 7, 16], [17, 0, 19, 6, 4, 1], [67, 0, 1, 4, 25, 13], [42, 0, 28, 2, 13, 11], [73, 8, 25, 0, 9, 12], [43, 0, 6, 14, 10, 26], [1, 8, 47, 11, 29, 29], [74, 5, 23, 16, 17, 14], [19, 4, 45, 13, 27, 11], [61, 1, 57, 0, 6, 25], [71, 8, 14, 10, 14, 11], [18, 2, 18, 10, 21, 22], [14, 0, 58, 7, 27, 31], [31, 5, 11, 3, 15, 31], [36, 7, 30, 15, 15, 6], [11, 0, 7, 15, 23, 25], [20, 6, 21, 2, 17, 21], [7, 6, 38, 13, 24, 7], [15, 3, 52, 4, 22, 2], [27, 3, 14, 6, 19, 18], [60, 6, 32, 9, 22, 9], [36, 0, 3, 2, 8, 27], [10, 8, 6, 16, 13, 20], [62, 0, 44, 0,